In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from datetime import datetime
from datetime import timedelta
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import support.ts_class as ts_class
import support.load_and_process_data as lpdata
import sys

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import Window

In [2]:
from importlib import reload
ts_class = reload(ts_class)
lpdata = reload(lpdata)

In [ ]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.pandas as ps

In [3]:
from deltalake import DeltaTable

In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
datasets_path = "/Users/esak/Projects/stock_traiding/nn_trader/datasets/"
features_data_path = "/Users/esak/Projects/stock_traiding/nn_trader/features_data/"

In [6]:
TODAY = datetime.today().strftime('%Y-%m-%d')
START_DATE = '1991-01-02 07:30:00'
END_DATE = '2022-09-30 23:59:59'

In [7]:
date_range = pd.DataFrame(data=[], index=pd.date_range(START_DATE,END_DATE, freq="min",name="Datetime")).reset_index()

In [8]:
def prepare_stock_data(source_data):

    ticker = DeltaTable(datasets_path+source_data).to_pandas().loc[:,["Datetime","Close","Volume","Ticker"]]
    ticker.loc[:,"Close"] = lpdata.detrend(ticker["Close"])
    #ticker.loc[:,"Volume"] = lpdata.detrend(ticker["Volume"])
    
    volume_name = ticker.loc[0,"Ticker"]+"_Volume"
    close_name = ticker.loc[0,"Ticker"]+"_Close"
    
    ticker = ticker.rename(columns={"Close":close_name, "Volume":volume_name})
    ticker = ticker.drop(columns=["Ticker"])
    
    return ticker
    #results.write.format("delta").mode("overwrite").save(train_data_path+data_name)

In [9]:
tickers = ["MSFT",'SP500','30Y_BOND','10Y_NOTE','GOLD']

In [12]:
#Add MinMax Scaler
def read_and_process_tickers(tickers):
    for source in tickers:
        print(source)
        stocks = prepare_stock_data(source)
        scaler = MinMaxScaler()
        scaler = scaler.fit(stocks.iloc[:,1:])
        stocks.iloc[:,1:]=scaler.transform(stocks.iloc[:,1:])
        stocks = date_range.merge(stocks, on="Datetime", how="left")
        stocks = stocks.ffill()
        stocks = stocks.fillna(0)
        stocks.to_parquet(features_data_path+source+".parquet", engine='pyarrow', index=False)
        #stocks["Datetime"] = pd.Series(stocks["Datetime"].dt.to_pydatetime(), dtype=object)
        #stocks_spark = ps.from_pandas(stocks).to_spark()
        #stocks_spark.write.format("delta").mode("overwrite").save(features_data_path+source)
        
    

In [13]:
read_and_process_tickers(tickers)

MSFT
SP500
30Y_BOND
10Y_NOTE
GOLD


In [15]:
econ_data =  DeltaTable(datasets_path+"economic_data").to_pandas()

In [16]:
distinct_events = list(set(econ_data["Event"]))

In [17]:
event_column_map = {}
count = 0

event = distinct_events[0]
event_column_map[event]=count

event_data = econ_data.loc[econ_data["Event"] == event,["Datetime","Actual"]]
event_data = event_data.rename(columns={"Actual":str(count)})

for event in distinct_events[1:]:
    count += 1
    event_data = event_data.merge(econ_data.loc[econ_data["Event"] == event,["Datetime","Actual"]], on="Datetime", how="outer")
    event_data = event_data.rename(columns={"Actual":str(count)})
    event_column_map[event]=count

In [18]:
event_data.head()

,Datetime,0,1,2,3,4,5,6,7,8,...,266,267,268,269,270,271,272,273,274,275
0,2021-05-12 07:30:00,442000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-427000.0,NaN
1,2021-05-05 07:30:00,-483000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7990000.0,NaN
2,2021-04-07 07:30:00,-60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3522000.0,NaN
3,2021-05-19 07:30:00,165000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1321000.0,NaN
4,2021-04-14 07:30:00,336000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5889000.0,NaN


In [19]:
event_data = event_data.sort_values(by="Datetime")

In [20]:
scaler = MinMaxScaler()
scaler = scaler.fit(event_data.iloc[:,1:])
event_data.iloc[:,1:]=scaler.transform(event_data.iloc[:,1:])

In [21]:
event_data.to_parquet(features_data_path+"econ_features.parquet", engine='pyarrow', index=False)

In [22]:
import json
with open(features_data_path+'econ_data_headers.json', 'w') as fp:
    json.dump(event_column_map, fp)

In [ ]:
def prepare_econ_train_data_old(events):
    for count, i in enumerate(events):
        event = i[0]
        sys.stdout.write('\r'+str(count)+' '+event)
        event_data = econ_data.filter(col("Event") == event).select(col("Datetime"),col("Actual"))
        event_data = date_range.join(event_data, on = "Datetime", how = "left")
        event_data = event_data.withColumn('Actual',F.last('Actual', ignorenulls=True).over(w_forward))
        event_data = event_data.fillna(0)
        event_data.coalesce(1)
        event = event.replace("(","").replace(")","").replace("&","").replace("/","").strip()
        event_data.write.format("delta").mode("overwrite").save(features_data_path+event)

In [24]:
event_column_map

{'Gasoline Production': 0,
 'Durable Goods Orders': 1,
 'Retail Sales Ex Gas/Autos (MoM)': 2,
 'KC Fed Composite Index': 3,
 'Real Personal Consumption (MoM)': 4,
 'Construction Spending': 5,
 'Capacity Utilization Rate': 6,
 '6-Month Bill Auction': 7,
 'Chain Store Sales (MoM)': 8,
 'Chicago PMI Prices': 9,
 'Manufacturing Production (MoM)': 10,
 'Factory orders ex transportation': 11,
 'Seevol Cushing Storage Report': 12,
 'Investing.com Gold Index': 13,
 'Pending Home Sales': 14,
 'Distillate Fuel Production': 15,
 'Core Durable Goods Orders (MoM)': 16,
 'Interest Rate Projection - Longer': 17,
 'Export Price Index': 18,
 'Housing Starts': 19,
 'Unit Labor Costs': 20,
 'S&P/CS HPI Composite - 20 s.a. (MoM)': 21,
 'Durables Excluding Transport (MoM)': 22,
 'QE Total': 23,
 'PCE price index (MoM)': 24,
 'API Weekly Crude Oil Stock': 25,
 'Goods Orders Non Defense Ex Air': 26,
 'Midwest Manufacturing': 27,
 'OPEC Crude oil Production Saudi Arabia': 28,
 'Gasoline Inventories': 29,
 'Tr